In [33]:
def main():
    import psycopg2
    import sys

    try:
        conn = psycopg2.connect("dbname='flight_info' user='postgres' host='localhost' password='test'")
    except:
        print('unable to connect')
        
    #cur=conn.cursor()
    
    command=1
    while (command!=0):
        command=int(input('enter number that does somthing, and 0 to quit '))
        
        
    

In [25]:
# Create a curson
cur = conn.cursor()

# Execute a statements and fetch results
try:
    cur.execute("SELECT name FROM student")
except:
    print "I can't SELECT from student"

# now let's fetch all the rows and print them
rows = cur.fetchall()
print "\nResults: \n"
for row in rows:
    print "   ", row


# now a query with more result columns
try:
    cur.execute("SELECT id, name, tot_cred FROM student ORDER BY name ASC")
except:
    print "I can't SELECT from student"

rows = cur.fetchall()
print "\nResults: \n"
for row in rows:
    print "   ", row
    print " or to access a particular column (2nd one):", row[1]

# close the connection
cur.close()

# close the connection
conn.close()


SyntaxError: Missing parentheses in call to 'print' (<ipython-input-25-1bf6a04765de>, line 8)

In [34]:
main()

unable to connect
enter number that does somthing, and 0 to quit 1
enter number that does somthing, and 0 to quit 1
enter number that does somthing, and 0 to quit 0
